<a href="https://colab.research.google.com/github/TienNguyen0712/student-alcohol-consumption-data-mining/blob/main/notebooks/04_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🎯 **Problem Formulation (Định nghĩa mục tiêu)**

**Mục tiêu:** Mục tiêu của bài toán là xây dựng mô hình hồi quy nhàm dự đoán điểm số **G3** của học sinh dựa trên các đặc trưng về thời gian học tập, sử dụng cồn, số buổi vắng học, ...

**Mục tiêu dự đoán:** Xác định các yếu tố ảnh hướng đến điểm só, hỗ trợ các giáo viên can thiếp sớm và cài thiện kết quả học tập

**Các mô hình được sử dụng:**
- Linear Regression
- Ridge / Lasso
- Random Forest Regressor
- Gradient Boosting

**Lý do lựa chọn mô hình được giải thích như sau:**
- Ridge / Lasso: Muốn mô hình có khả năng trực quan cũng như muốn giải thích đánh giá xem đặc trưng nào đang ảnh hưởng đến với điểm số
- Linear Regression: Tương tự như với Ridge và Lasso cũng xem và đánh giá các mối quan hệ của các đặc trưng tuyến tính cso trong bài
- Random Forest / Gradient Boosting: Việc sử dụng 2 mô hình này nhằm đánh giá bộ dữ liệu thông qua tree-base cũng giúp đánh giá xem đặc trưng nào quan trọng và đang có yếu tố narh hưởng đến điểm sô


Mô hình được đánh giá dựa theo cac chỉ số **MAE**, **RMSE**, **R^2**, trong đó ưu tiên MAE vì đơn vị điếm số thực tế

In [87]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

In [88]:
student_merge = pd.read_csv('student_merge.csv')
student_merge.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,traveltime,studytime,failures,famrel,goout,Dalc,Walc,health,absences,freetime
0,GP,F,15,R,GT3,T,1,1,at_home,other,...,2.0,4.0,0.5,3.0,2.0,1.0,1.0,1.0,4.0,1.0
1,GP,F,15,R,GT3,T,1,1,other,other,...,3.0,1.0,1.0,5.0,5.0,1.0,1.0,1.0,2.0,5.0
2,GP,F,15,R,GT3,T,1,1,other,other,...,1.0,2.0,1.0,3.0,4.0,2.0,4.0,5.0,2.0,3.0
3,GP,F,15,R,GT3,T,2,2,at_home,other,...,1.0,1.0,0.0,4.0,1.0,1.0,1.0,2.0,8.0,3.0
4,GP,F,15,R,GT3,T,2,4,services,health,...,1.0,3.0,0.0,4.0,2.0,1.0,1.0,5.0,2.0,3.0


## 🔀 **Train – Validation – Test Split (Chia tập huấn luyện và đánh giá)**

In [89]:
student_merge.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'nursery', 'internet', 'G3', 'G2', 'G1',
       'guardian', 'schoolsup', 'famsup', 'paid', 'activities', 'higher',
       'romantic', 'traveltime', 'studytime', 'failures', 'famrel', 'goout',
       'Dalc', 'Walc', 'health', 'absences', 'freetime'],
      dtype='object')

In [118]:
drop_columns = ['sex', 'age', 'address', 'famsize', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'G3', 'guardian', 'schoolsup', 'famsup']

target = student_merge['G3']
features = student_merge.drop(columns=drop_columns)

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

X_train.shape, X_test.shape

((545, 21), (137, 21))

In [91]:
num_freatures = features.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_features = features.select_dtypes(include=['object']).columns.tolist()

print("Num features: ", num_freatures)
print("Cat features: ", cat_features)

Num features:  ['G2', 'G1', 'traveltime', 'studytime', 'failures', 'famrel', 'goout', 'Dalc', 'Walc', 'health', 'absences', 'freetime']
Cat features:  ['school', 'Pstatus', 'reason', 'nursery', 'internet', 'paid', 'activities', 'higher', 'romantic']


In [92]:
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")) # Không sinh thứ tự giả, an toàn
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_freatures),
        ("cat", categorical_transformer, cat_features),
    ],
    remainder="drop"
)

## 🤖 **Model Selection (Lựa chọn mô hình)**

In [93]:
ridge = Ridge(
    alpha=1.0,
    fit_intercept=True,
    solver='auto',
    random_state=42
)

pipe_ridge = Pipeline(steps=[
    ("preprocess", preprocess),
    ("ridge", ridge)
])

In [94]:
lasso = Lasso(
    alpha=1.0,
    max_iter=1000,
    selection='cyclic'
)

pipe_lasso = Pipeline(steps=[
    ("preprocess", preprocess),
    ("lasso", lasso)
])

In [95]:
linear_reg = LinearRegression(
    fit_intercept=True,
    copy_X=True,
    n_jobs=None,
    positive=False
)

pipe_linear_reg = Pipeline(steps=[
    ("preprocess", preprocess),
    ("linear_reg", linear_reg)
])

In [96]:
forest_reg = RandomForestRegressor(
    n_estimators=300,
    max_depth=5,
    min_samples_leaf=10,
    max_features="sqrt",
    random_state=42,
    n_jobs=-1
)

pipe_forest_reg = Pipeline(steps=[
    ("preprocess", preprocess),
    ("forest_reg", forest_reg)
])

In [97]:
gb_reg = GradientBoostingRegressor(
    learning_rate=0.05,
    n_estimators=300,
    max_depth=3,
    subsample=0.8,
    min_samples_leaf=10,
    random_state=42
)

pipe_gb_reg = Pipeline(steps=[
    ("preprocess", preprocess),
    ("gb_reg", gb_reg)
])

## 🏋️**Model Training (Huấn luyện mô hình)**

In [98]:
pipe_ridge.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['G2', 'G1', 'traveltime',
                                                   'studytime', 'failures',
                                                   'famrel', 'goout', 'Dalc',
                                                   'Walc', 'health', 'absences',
                                                   'freetime']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['school', 'Pstatus',
                                                   'reason', 'nursery',
                                                   'internet', 'paid',
                                                   'activities', 'higher',
                                                   'romantic'])])),
                ('ridge', Ridge(random_state=42))])

In [99]:
pipe_lasso.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['G2', 'G1', 'traveltime',
                                                   'studytime', 'failures',
                                                   'famrel', 'goout', 'Dalc',
                                                   'Walc', 'health', 'absences',
                                                   'freetime']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['school', 'Pstatus',
                                                   'reason', 'nursery',
                                                   'internet', 'paid',
                                                   'activities', 'higher',
                                                   'romantic'])])),
                ('lasso', Lasso())])

In [100]:
pipe_linear_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['G2', 'G1', 'traveltime',
                                                   'studytime', 'failures',
                                                   'famrel', 'goout', 'Dalc',
                                                   'Walc', 'health', 'absences',
                                                   'freetime']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['school', 'Pstatus',
                                                   'reason', 'nursery',
                                                   'internet', 'paid',
                                                   'activities', 'higher',
                                                   'romantic'])])),
                ('linear_reg', LinearRegression())])

In [101]:
pipe_forest_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['G2', 'G1', 'traveltime',
                                                   'studytime', 'failures',
                                                   'famrel', 'goout', 'Dalc',
                                                   'Walc', 'health', 'absences',
                                                   'freetime']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['school', 'Pstatus',
                                                   'reason', 'nursery',
                                                   'internet', 'paid',
                                                   'activities', 'higher',
                                                   'romantic'])])),
                ('forest_reg',
                 RandomForestRegressor(max_depth=5, max_features='sqrt',
                                       min_samples_leaf=10, n_estimators=300,
                                       n_jobs=-1, random_state=42))])

In [102]:
pipe_gb_reg.fit(X_train, y_train)

Pipeline(steps=[('preprocess',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['G2', 'G1', 'traveltime',
                                                   'studytime', 'failures',
                                                   'famrel', 'goout', 'Dalc',
                                                   'Walc', 'health', 'absences',
                                                   'freetime']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['school', 'Pstatus',
                                                   'reason', 'nursery',
                                                   'internet', 'paid',
                                                   'activities', 'higher',
                                                   'romantic'])])),
                ('gb_reg',
                 GradientBoostingRegressor(learning_rate=0.05,
                                           min_samples_leaf=10,
                                           n_estimators=300, random_state=42,
                                           subsample=0.8))])

## 📊 **Model Evaluation (Đánh giá mô hình)**

In [103]:
y_pred_ridge = pipe_ridge.predict(X_test)

mae_ridge = mean_absolute_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mean_squared_error(y_test, y_pred_ridge))
r2_ridge = r2_score(y_test, y_pred_ridge)

In [104]:
y_pred_lasso = pipe_lasso.predict(X_test)

mae_lasso = mean_absolute_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mean_squared_error(y_test, y_pred_lasso))
r2_lasso = r2_score(y_test, y_pred_lasso)

In [105]:
y_pred_linear_reg = pipe_linear_reg.predict(X_test)

mae_linear_reg = mean_absolute_error(y_test, y_pred_linear_reg)
rmse_linear_reg = np.sqrt(mean_squared_error(y_test, y_pred_linear_reg))
r2_linear_reg = r2_score(y_test, y_pred_linear_reg)

In [106]:
y_pred_forest_reg = pipe_forest_reg.predict(X_test)

mae_forest_reg = mean_absolute_error(y_test, y_pred_forest_reg)
rmse_forest_reg = np.sqrt(mean_squared_error(y_test, y_pred_forest_reg))
r2_forest_reg = r2_score(y_test, y_pred_forest_reg)

In [107]:
y_pred_gb_reg = pipe_gb_reg.predict(X_test)

mae_gb_reg = mean_absolute_error(y_test, y_pred_gb_reg)
rmse_gb_reg = np.sqrt(mean_squared_error(y_test, y_pred_gb_reg))
r2_gb_reg = r2_score(y_test, y_pred_gb_reg)

## **🏆 Model Comparison (So sánh mô hình)**

In [108]:
from sklearn.model_selection import KFold, cross_validate

cv = KFold(n_splits=5, shuffle=True, random_state=42)

scoring = {
    "rmse": "neg_root_mean_squared_error",
    "mae": "neg_mean_absolute_error",
    "r2": "r2",
}

models = {
    "Ridge": pipe_ridge,
    "Lasso": pipe_lasso,
    "Linear Regression": pipe_linear_reg,
    "Random Forest": pipe_forest_reg,
    "Gradient Boosting": pipe_gb_reg,
}

results = []

for name, model in models.items():
    cv_out = cross_validate(
        model,
        X_train, y_train,
        cv=cv,
        scoring=scoring,
        n_jobs=-1,
        return_train_score=False
    )

    rmse = -cv_out["test_rmse"]
    mae  = -cv_out["test_mae"]
    r2   = cv_out["test_r2"]

    results.append({
        "model": name,
        "RMSE_mean": rmse.mean(),
        "RMSE_std": rmse.std(),
        "MAE_mean": mae.mean(),
        "MAE_std": mae.std(),
        "R2_mean": r2.mean(),
        "R2_std": r2.std(),
    })

results_sorted = sorted(results, key=lambda d: d["RMSE_mean"])

for r in results_sorted:
    print(
        f"{r['model']:16s} | "
        f"RMSE: {r['RMSE_mean']:.4f} ± {r['RMSE_std']:.4f} | "
        f"MAE:  {r['MAE_mean']:.4f} ± {r['MAE_std']:.4f} | "
        f"R2:   {r['R2_mean']:.4f} ± {r['R2_std']:.4f}"
    )


Ridge            | RMSE: 1.2851 ± 0.2291 | MAE:  0.8193 ± 0.0810 | R2:   0.8607 ± 0.0485
Linear Regression | RMSE: 1.2856 ± 0.2295 | MAE:  0.8208 ± 0.0808 | R2:   0.8606 ± 0.0485
Gradient Boosting | RMSE: 1.2924 ± 0.2252 | MAE:  0.8942 ± 0.0889 | R2:   0.8597 ± 0.0441
Lasso            | RMSE: 1.6117 ± 0.2053 | MAE:  1.0976 ± 0.1050 | R2:   0.7841 ± 0.0521
Random Forest    | RMSE: 1.9716 ± 0.1425 | MAE:  1.2852 ± 0.0657 | R2:   0.6807 ± 0.0353


In [109]:
# Ridge
print("MAE: ", mae_ridge)
print("RMSE: ", rmse_ridge)
print("R^2: ", r2_ridge)

MAE:  0.8044159341014333
RMSE:  1.2140835404840848
R^2:  0.8306554264640771


In [110]:
# Lasso
print("MAE: ", mae_lasso)
print("RMSE: ", rmse_lasso)
print("R^2: ", r2_lasso)

MAE:  0.949351587406748
RMSE:  1.4362647520570344
R^2:  0.7630028372456705


In [111]:
# Linear Regression
print("MAE: ", mae_linear_reg)
print("RMSE: ", rmse_linear_reg)
print("R^2: ", r2_linear_reg)

MAE:  0.8057605827656287
RMSE:  1.2156717200041849
R^2:  0.8302120871301493


In [112]:
# Random Forest
print("MAE: ", mae_forest_reg)
print("RMSE: ", rmse_forest_reg)
print("R^2: ", r2_forest_reg)

MAE:  1.0673853528100303
RMSE:  1.5248884469713713
R^2:  0.7328530069827617


In [113]:
# Gradient Boosting
print("MAE: ", mae_gb_reg)
print("RMSE: ", rmse_gb_reg)
print("R^2: ", r2_gb_reg)

MAE:  0.8911607854056752
RMSE:  1.3239537096904177
R^2:  0.7986184201227337


## **⚙️ Hyperparameter Tuning (Tổi ưu hóa tham số)**

In [114]:
from sklearn.model_selection import GridSearchCV, KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42)

param_grid = {
    "ridge__alpha": np.logspace(-4, 4, 30)
}

gs = GridSearchCV(
    estimator=pipe_ridge,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=cv,
    n_jobs=-1
)

gs.fit(X_train, y_train)

print("Best params:", gs.best_params_)
print("Best CV RMSE:", -gs.best_score_)

best_ridge = gs.best_estimator_

Best params: {'ridge__alpha': np.float64(4.893900918477489)}
Best CV RMSE: 1.2842199419070508


## 🔎 **Feature Importance (Đặc trưng quan trọng)**

In [115]:
best_ridge.fit(X_train, y_train)
preprocess = best_ridge.named_steps["preprocess"]
feature_names = preprocess.get_feature_names_out()

In [116]:
coefs = best_ridge.named_steps["ridge"].coef_

In [117]:
coef_df = pd.DataFrame({
    "feature": feature_names,
    "coef": coefs,
    "abs_coef": np.abs(coefs)
})

coef_df = coef_df.sort_values("abs_coef", ascending=False)
coef_df.head(15)

,feature,coef,abs_coef
0,num__G2,2.828142,2.828142
1,num__G1,0.440448,0.440448
16,cat__reason_course,0.146432,0.146432
27,cat__activities_yes,-0.115662,0.115662
26,cat__activities_no,0.115662,0.115662
17,cat__reason_home,-0.111766,0.111766
4,num__failures,-0.106732,0.106732
14,cat__Pstatus_A,0.099052,0.099052
15,cat__Pstatus_T,-0.099052,0.099052
24,cat__paid_no,0.097191,0.097191


## 📝 **Conclusion (Kết luận)**

**Insight**

Quan trọng nhất rất rõ ràng có thể thấy là điểm G1 - G2 gần như quyết định đến điểm cuối cùng, đây giống với bài toàn dự đoán tương lai gần hơn so với dự đoán trong quá khá

Lý do học tập như học vì khóa học điểm sẽ cao hơn học ở gần nhà sau khi đã kiểm soát G1 và G2. Đây chính là hành vi không phải ngẫu nhiên

Việc tham gia các hoạt động ngoại khóa điểm sẽ thấ còn không tham gia thì điểm cao hơn. Không phải do hoạt động mà là học sinh yếu mới cần hỗ trợ

Cacs nhóm như sử dụng cồn hằng ngày và cuối tuần hay số lần thất bauh ở với người thân sẽ có xu hướng điều chỉnh và xem xét